# Split Data
- Splits the TIMIT dataset into a `TRAIN`, `VALIDATION`, `TEST` dataset
- Note: the TIMIT dataset already comes with a `TEST` folder so we will simply take a % of the `TEST` dataset for validation

## Step 1) Import Audio
- Create Training Dataset
- Create Test Dataset
- Since TIMIT does not differentiate between Test and Validation, we will manually split the Test into Test and Validation as a downstream task to respect the original naming conventions of the TIMIT dataset

### Motivation
The motivation for creating this dataset is because the existing `timit_asr` Hugging Face datasets do not use the complete TIMIT `TEST` and `TRAIN` data available. 

In addition, we also add  speaker gender and duration of speech.

### Linguistic Distinction in categories
`SA` = "Speaker Accent"/Dialect or Shibboleth sentences designed to highlight dialect region differences
`SX` = Phonetically Compact sentences designed to highlight pairs of phones of interest (i.e. voiced vs unvoiced velar stops) in specific phonetic contexts (i.e. coda position = at the end of a word)
`SI` = Phonetically Diverse sentences designed to highlight many different phonemes and sentence types per speaker

In [5]:
from datasets import Dataset, Audio, DatasetDict, load_dataset
import os

/Users/kyleng/B_Organized/A_School/A_CSCI_467/Final_Proj/code/IPA-ASR/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from timit_metadata_extractors import get_transcription_detail, get_speech_duration, get_replace_ending, get_timit_path, get_speaker_info, get_sentence_info, get_text, get_ipa_transcription

In [2]:
PATH_TO_TIMIT = "../data/input_data/TIMIT-Database/TIMIT"
UPLOAD_TIMIT_BASE_NAME= "kylelovesllms/timit_asr"

In [6]:
def get_audio_files(dir: str, file_type: str = "wav") -> dict[tuple[str, list[str]], tuple[str, list[str]]]:
    """
    Walks through every directory in `dir` and returns all files which end in `file_type`
    """

    audio_paths = []

    # Walk through each directory
    for dirpath, _dirnames, filenames in os.walk(dir):
        # Check each file in the directory
        for file_name in filenames:
            # If the `file_type` matches
            if file_name.endswith(file_type):
                # Add that file
                full_local_path = os.path.join(dirpath, file_name)
                audio_paths.append(full_local_path)
    return audio_paths

In [7]:
# Quick sanity check that we get the appropriate local paths and task names
paths = get_audio_files("../data/input_data/TIMIT-Database/TIMIT/TEST")
print("local paths", paths[:3])

local paths ['../data/input_data/TIMIT-Database/TIMIT/TEST/DR4/MGMM0/SX139.wav', '../data/input_data/TIMIT-Database/TIMIT/TEST/DR4/MGMM0/SA2.wav', '../data/input_data/TIMIT-Database/TIMIT/TEST/DR4/MGMM0/SX229.wav']


In [8]:
train_path = os.path.join(PATH_TO_TIMIT, "TRAIN")
testvalidation_path = os.path.join(PATH_TO_TIMIT, "TEST")

In [18]:
train_dataset = Dataset.from_dict(
    {
        "audio": [audio_path for audio_path in get_audio_files(train_path)]
    }
).cast_column("audio", Audio())

testvalidation_dataset = Dataset.from_dict(
     {
        "audio": [audio_path for audio_path in get_audio_files(testvalidation_path)]
    }
).cast_column("audio", Audio())

# TODO REMOVE AFTER DEBUGGING
# train_dataset = Dataset.from_dict(
#     {
#         "audio": [audio_path for audio_path in get_audio_files(train_path)][:1]
#     }
# ).cast_column("audio", Audio())

# testvalidation_dataset = Dataset.from_dict(
#      {
#         "audio": [audio_path for audio_path in get_audio_files(testvalidation_path)][:1]
#     }
# ).cast_column("audio", Audio())

In [8]:
train_dataset

Dataset({
    features: ['audio'],
    num_rows: 3629
})

In [9]:
# Quick sanity check to make sure we have valid audio files
train_dataset[0]["audio"]

{'path': '../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SI681.wav',
 'array': array([-2.13623047e-04,  6.10351562e-05,  3.05175781e-05, ...,
        -3.05175781e-05, -9.15527344e-05, -6.10351562e-05]),
 'sampling_rate': 16000}

## Step 2) Add Metadata
Although not all of the metadata is needed for this specific project, perhaps others in the open-source community will find this metadata helpful

In [10]:
def add_metadata(example):
    """
    Adds transcriptions and metadata to the TIMIT dataset

    Note: after using `example["audio"]`, the `example["audio"]["path"]` is 
    automatically set to None for security reasons
    (see Github Issue: https://github.com/huggingface/datasets/issues/5190)
    """
    wav_path = example["audio"]["path"]

    # Add Transcriptions
    phn_file = get_replace_ending(wav_path, new_extension=".phn")
    wrd_file = get_replace_ending(wav_path, new_extension=".wrd")
    text_file = get_replace_ending(wav_path, new_extension=".txt")

    example["phonetic_detail"] = get_transcription_detail(phn_file)
    example["word_detail"] = get_transcription_detail(wrd_file)
    example["text"] = get_text(text_file)

    # Speech Duration
    example["duration"] = get_speech_duration(
        example["audio"]["array"], sr=example["audio"]["sampling_rate"])

    # TIMIT Path
    example["timit_path"] = get_timit_path(
        abs_path=wav_path, base_path=PATH_TO_TIMIT)

    # Speaker Metadata
    (dialect_region, dialect_region_name), (speaker_id, sex) = get_speaker_info(
        example["timit_path"])
    example["dialect_region"] = dialect_region
    example["dialect_region_name"] = dialect_region_name
    example["speaker_id"] = speaker_id
    example["speaker_sex"] = sex

    # Sentence Metadata
    sentence_id, sentence_type = get_sentence_info(wav_path)
    example["id"] = sentence_id
    example["sentence_type"] = sentence_type
    return example

In [11]:
train_dataset_with_metadata = train_dataset.map(add_metadata)
testvalidation_dataset_with_metadata = testvalidation_dataset.map(add_metadata)

Map: 100%|██████████| 1340/1340 [00:31<00:00, 42.04 examples/s]


## Step 3) Upload Dataset to HuggingFace Hub
- Note: we will upload the `testvalidation_dataset_with_metadata` as `test` in the TIMIT tradition
- We will also create a train/validation/test split and have more features extracted from the data but do so in a different repository to remain consistent with the original TIMIT dataset

In [12]:
timit_asr_dataset_base = DatasetDict({
    "train": train_dataset_with_metadata,
    "test": testvalidation_dataset_with_metadata
})

In [13]:
timit_asr_dataset_base

DatasetDict({
    train: Dataset({
        features: ['audio', 'phonetic_detail', 'word_detail', 'text', 'duration', 'timit_path', 'dialect_region', 'dialect_region_name', 'speaker_id', 'speaker_sex', 'id', 'sentence_type'],
        num_rows: 3629
    })
    test: Dataset({
        features: ['audio', 'phonetic_detail', 'word_detail', 'text', 'duration', 'timit_path', 'dialect_region', 'dialect_region_name', 'speaker_id', 'speaker_sex', 'id', 'sentence_type'],
        num_rows: 1340
    })
})

#### Push Upload and Verify

In [ ]:
timit_asr_dataset_base.push_to_hub(UPLOAD_TIMIT_BASE_NAME)

Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.95s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/kylelovesllms/timit_asr/commit/29c837c76686cb61f3efe199b0ea7d959b12f343', commit_message='Upload dataset', commit_description='', oid='29c837c76686cb61f3efe199b0ea7d959b12f343', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kylelovesllms/timit_asr', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kylelovesllms/timit_asr'), pr_revision=None, pr_num=None)

In [22]:
timit_asr_dataset_base_from_hub = load_dataset(UPLOAD_TIMIT_BASE_NAME)

Generating test split: 100%|██████████| 1340/1340 [00:00<00:00, 9634.24 examples/s]


In [ ]:
# Verify that the repository is pulled down correctly
print(timit_asr_dataset_base_from_hub)

DatasetDict({
    train: Dataset({
        features: ['audio', 'phonetic_detail', 'word_detail', 'text', 'duration', 'timit_path', 'dialect_region', 'dialect_region_name', 'speaker_id', 'speaker_sex', 'id', 'sentence_type'],
        num_rows: 3629
    })
    test: Dataset({
        features: ['audio', 'phonetic_detail', 'word_detail', 'text', 'duration', 'timit_path', 'dialect_region', 'dialect_region_name', 'speaker_id', 'speaker_sex', 'id', 'sentence_type'],
        num_rows: 1340
    })
})
{'audio': {'path': None, 'array': array([-2.13623047e-04,  6.10351562e-05,  3.05175781e-05, ...,
       -3.05175781e-05, -9.15527344e-05, -6.10351562e-05]), 'sampling_rate': 16000}, 'phonetic_detail': [{'start': 0, 'stop': 1960, 'utterance': 'h#'}, {'start': 1960, 'stop': 2466, 'utterance': 'w'}, {'start': 2466, 'stop': 3480, 'utterance': 'ix'}, {'start': 3480, 'stop': 4000, 'utterance': 'dcl'}, {'start': 4000, 'stop': 5960, 'utterance': 's'}, {'start': 5960, 'stop': 7480, 'utterance': 'ah'}, {'st

## Step 4) Adding Phonetic Transcriptions
- Although the TIMIT IPA has a `phonetic_detail` which is documented in `TIMIT/DOC/PHONCODE.DOC`, some downstream use cases may require the use of transcription records in the International Phonetic Alphabet (IPA) format
- This format is universal accross almost every tradition in phonetics 
- Note: the transcription scheme used in the repository below is not peer reviewed and is specific to a broad transcription for evaluating a fine tuned Wav2Vec2 model against Wav2Vec2-XLSR

In [ ]:
def add_ipa_transcription(example):
    """
    Returns an interpretation of the IPA transcription in the same
    format as `phonetic_detail`
    """
    wav_path = example["audio"]["path"]

    # Add Transcriptions
    phn_file = get_replace_ending(wav_path, new_extension=".phn")
    wrd_file = get_replace_ending(wav_path, new_extension=".wrd")
    text_file = get_replace_ending(wav_path, new_extension=".txt")

    example["phonetic_detail"] = get_transcription_detail(phn_file)
    example["word_detail"] = get_transcription_detail(wrd_file)

    example["text"] = get_text(text_file)
    example["ipa_transcription"] = get_ipa_transcription(example["phonetic_detail"])

    # Speech Duration
    example["duration"] = get_speech_duration(
        example["audio"]["array"], sr=example["audio"]["sampling_rate"])

    # TIMIT Path
    example["timit_path"] = get_timit_path(
        abs_path=wav_path, base_path=PATH_TO_TIMIT)

    # Speaker Metadata
    (dialect_region, dialect_region_name), (speaker_id, sex) = get_speaker_info(
        example["timit_path"])
    example["dialect_region"] = dialect_region
    example["dialect_region_name"] = dialect_region_name
    example["speaker_id"] = speaker_id
    example["speaker_sex"] = sex

    # Sentence Metadata
    sentence_id, sentence_type = get_sentence_info(wav_path)
    example["id"] = sentence_id
    example["sentence_type"] = sentence_type

    return example

In [20]:
train_dataset_with_ipa = train_dataset.map(add_ipa_transcription)

Map:   3%|▎         | 95/3629 [00:00<00:07, 470.49 examples/s]

../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SI681.wav
missing closure of symbol t
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SA2.wav
missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SX411.wav
missing closure of symbol b
missing closure of symbol g
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SA1.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SX231.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SX51.wav
missing closure of symbol p
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SX141.wav
missing closure of symbol p
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SI1941.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SI1311.wav
missing closure of symbol b
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMDM0/SX321.wav
missing closure of symbol k
missing closure of symbol b
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MCSS0/SX300.wav
missing closure of symbol

Map:   4%|▍         | 144/3629 [00:00<00:07, 471.32 examples/s]

../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/FLKM0/SA1.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/FLKM0/SX350.wav
missing closure of symbol b
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/FLKM0/SX80.wav
missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/FLKM0/SX440.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/FLKM0/SI1880.wav
missing closure of symbol t
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMGC0/SX315.wav
missing closure of symbol b
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMGC0/SA2.wav
missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMGC0/SX405.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMGC0/SA1.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMGC0/SX225.wav
missing closure of symbol k
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMGC0/SX45.wav
missing closure of symbol t
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR4/MMGC0/SI1305.wav
../data/input_data

Map:   7%|▋         | 243/3629 [00:00<00:07, 481.26 examples/s]

...

Map:  98%|█████████▊| 3543/3629 [01:17<00:00, 189.37 examples/s]

../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MKLW0/SX131.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRCG0/SX258.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRCG0/SX438.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRCG0/SI2058.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRCG0/SA2.wav
missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRCG0/SA1.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRCG0/SX348.wav
missing closure of symbol p
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRCG0/SX78.wav
missing closure of symbol d
missing closure of symbol t
missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRCG0/SI798.wav
missing closure of symbol d
missing closure of symbol b
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRCG0/SI1428.wav
missing closure of symbol d
missing closure of symbol t
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRCG0/SX168.wav
missing closure of symbol

Map:  99%|█████████▉| 3587/3629 [01:17<00:00, 223.99 examples/s]

missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRDD0/SX60.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRDD0/SA1.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRDD0/SI2310.wav
missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRDD0/SX150.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRDD0/SX330.wav
missing closure of symbol k
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MRDD0/SX240.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MKLS0/SX267.wav
missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MKLS0/SI1533.wav
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MKLS0/SI1437.wav
missing closure of symbol t
missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MKLS0/SA2.wav
missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MKLS0/SX177.wav
missing closure of symbol d
../data/input_data/TIMIT-Database/TIMIT/TRAIN/DR1/MK

Map: 100%|██████████| 3629/3629 [01:31<00:00, 39.63 examples/s] 


In [12]:
print(train_dataset_with_ipa[0]["text"])

Would such an act of refusal be useful?


In [13]:
print([seg["utterance"] for seg in train_dataset_with_ipa[0]["phonetic_detail"]])

['h#', 'w', 'ix', 'dcl', 's', 'ah', 'tcl', 'ch', 'ix', 'n', 'ae', 'kcl', 't', 'ix', 'v', 'r', 'ix', 'f', 'y', 'ux', 'zh', 'el', 'bcl', 'b', 'iy', 'y', 'ux', 's', 'f', 'el', 'h#']


In [15]:
print(train_dataset_with_ipa[0]["ipa_transcription"])

['w', 'ɪ', 'd', 's', 'ʌ', 't', 'tʃ', 'ɪ', 'n', 'æ', 'k', 't', 'ɪ', 'v', 'r', 'ɪ', 'f', 'j', 'u', 'ʒ', 'l', 'b', 'i', 'j', 'u', 's', 'f', 'l']
